In [44]:
import pandas as pd 
import json
import os
import psycopg2
from configparser import ConfigParser
from config import config
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
#This function facilitates the connection to the database
def connect():
    try:
        
        parser = ConfigParser()
        parser.read('database.ini')

        #params = config()
        host = parser.get('postgresql','DB_HOST')
        port = parser.get('postgresql','DB_PORT')
        database =parser.get('postgresql','DB_NAME')
        user = parser.get('postgresql','DB_USER')
        password = parser.get('postgresql','DB_PW')

        #params = config()
        #print(host, port, password, user, database)
        conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

        curr = conn.cursor()
        
        query = """SELECT table_name from information_schema.tables WHERE table_schema = 'public'"""
        curr.execute(query)
        
        rows = curr.fetchall()

        for row in rows:
            print(row[0])
    except( Exception, psycopg2.DatabaseError) as err:
        print(err)
    finally:
        if conn is not None:
            conn.close()
            curr.close()
    


users
watchlist_gpu
watchlist_cpu
cpus
gpus
device_tokens
review_sentiments


In [ ]:
def getUsers(curr):
    

In [28]:
#This function sorts the cpu products by renaming and merging columns for consistency
def filter_cpu(data):
    data_des = data[['type', 'retailer', 'brand','# of CPU Cores', '# of Threads', 'Base Clock', 'Max Boost Clock',
                                        'Default TDP / TDP', 'Processor Base Frequency','Max Turbo Frequency', 'TDP']]
    
    df_change = data_des.loc[data_des['Base Clock'].isnull()]
    data_des.drop(df_change.index, inplace=True)
    df_change.drop(['Base Clock', 'Default TDP / TDP', 'Max Boost Clock'], axis=1, inplace=True)
    df_change.rename({'Processor Base Frequency': 'Base Clock', 'Max Turbo Frequency': 'Max Boost Clock'}, axis=1, inplace=True)
    
    data_des.drop(['Processor Base Frequency', 'Max Turbo Frequency', 'TDP'], axis=1, inplace=True)
    data_des.rename({'Default TDP / TDP': 'TDP'}, axis=1, inplace=True)
    data_des = pd.merge(data_des, df_change, how='outer')
    return data_des


In [ ]:
def main():
    #get users
    #get users wishlist; filter
    #get all products that are not in wishlist; filter
    #encode data 
    #predict